# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Откройте файл с данными и изучите общую информацию

In [137]:
import pandas as pd
from pymystem3 import Mystem # когда дошел до лемматизации, решил, что импортировать библиотеку лучше тут
m = Mystem() 
credit_score = pd.read_csv('/datasets/data.csv')
credit_score.head(10)


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [138]:
credit_score.info() #выводим информаию о таблице

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


**Вывод**  

В days_employed и total_income одинаковые значения ( вероятно потому, что при подсчёте общего дохода, при отуствии работающих дней количество общего дохода не учитывается в total_income) Тип данных в порядке, исключение - неудобный формат для  total_income. Надо будет окргулить и сделать int. Надо избавиться от отрицательных значений в  days_employed. И в education проблемы с регистром.

## Шаг 2. Предобработка данных

### Обработка пропусков

In [139]:
print(credit_score.isna().sum())

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64


In [140]:
crdis_filterd = credit_score[credit_score['total_income'].isna()] #ищем пропущенные значения в total_income и выводим первые 10 строк на экран
crdis_filterd.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
65,0,NaN,21,среднее,1,Не женат / не замужем,4,M,компаньон,0,NaN,операции с коммерческой недвижимостью
67,0,NaN,52,высшее,0,женат / замужем,0,F,пенсионер,0,NaN,покупка жилья для семьи
72,1,NaN,32,высшее,0,женат / замужем,0,M,госслужащий,0,NaN,операции с коммерческой недвижимостью
82,2,NaN,50,высшее,0,женат / замужем,0,F,сотрудник,0,NaN,жилье
83,0,NaN,52,среднее,1,женат / замужем,0,M,сотрудник,0,NaN,жилье


In [141]:
income_grouped_mean = credit_score.groupby('income_type')['total_income'].mean() 
print(income_grouped_mean) #посчитаем среднее значение и занесем в переменную income_grouped_mean
income_grouped_median = credit_score.groupby('income_type')['total_income'].median()
print(income_grouped_median)#посчитаем медиану и занесем в переменную income_grouped_median


income_type
безработный        131339.751676
в декрете           53829.130729
госслужащий        170898.309923
компаньон          202417.461462
пенсионер          137127.465690
предприниматель    499163.144947
сотрудник          161380.260488
студент             98201.625314
Name: total_income, dtype: float64
income_type
безработный        131339.751676
в декрете           53829.130729
госслужащий        150447.935283
компаньон          172357.950966
пенсионер          118514.486412
предприниматель    499163.144947
сотрудник          142594.396847
студент             98201.625314
Name: total_income, dtype: float64


Вывод: данные среднего и медианы совпадают, можно избавляться от пропусков

In [142]:
def fillbygroup(data, row): # функция заменяет NaN в row на медианное значение этого столбца у соответствующего income_type
    unique_inc_type = credit_score['income_type'].unique()
    for type in unique_inc_type:
        data.loc[data['income_type'] == type, row] = data.loc[data['income_type'] == type, row].fillna(data[data['income_type'] == type]['total_income'].median())
    return data

credit_score = fillbygroup(credit_score, 'total_income')


In [143]:
credit_score.info() #вызываем метод проверить заполнение пропусков

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [144]:
credit_score['days_employed'] = credit_score['days_employed'].apply(abs) # избавляемся от отрицательных значений
credit_score = fillbygroup(credit_score, 'days_employed') # теперь можно применить функцию 

dob_mean = credit_score['dob_years'].mean() # в возрасте нет выдающихся значений, поэтому заполняем нули средним возрастом
credit_score.loc[credit_score['dob_years'] == 0, 'dob_years'] = dob_mean
credit_score['dob_years'] = credit_score['dob_years'].round().astype('int')
print(credit_score.info()) #снова проверяем удаление пропусков


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB
None


**Вывод** Пропуски были исключены, можно работать дальше

### Замена типа данных

In [145]:
credit_score['days_employed'] = credit_score['days_employed'].round().astype('int') # округляю float, переводим переведем в int
credit_score['total_income'] = credit_score['total_income'].round().astype('int')# повторяем действие 
#Сумма не будет считаеться из-за наличия строковых значений total_income

In [146]:
credit_score.info() # проверяем изменение значений

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


In [147]:
credit_score['education'] = credit_score['education'].str.lower()
credit_score['education'] #привел значения в education к нижнему регистру

0         высшее
1        среднее
2        среднее
3        среднее
4        среднее
          ...   
21520    среднее
21521    среднее
21522    среднее
21523    среднее
21524    среднее
Name: education, Length: 21525, dtype: object

**Вывод** Данные заменяли для подсчёта 

### Обработка дубликатов

In [148]:
print(credit_score.duplicated().sum()) #71 дубликат, необходимо удалить

71


In [149]:
credit_score = credit_score.drop_duplicates().reset_index() #удаляем дубликаты, ресетим индексы
print(credit_score.duplicated().sum())

0


**Вывод** Дубликаты удалены. Хотя мне кажется я как-то просто отделался и ощущение, что что-то упуcтил :D

### Лемматизация

In [150]:
def create_category_purpose(row): 
        lemmas = m.lemmatize(row['purpose'])
        if 'свадьба' in lemmas:
            return ' на свадьбу'
        if 'жилье' in lemmas or 'недвижимость' in lemmas:
            return 'на недвижимость'
        if 'автомобиль' in lemmas:
            return 'на автомобиль'
        if 'образование' in lemmas:
            return 'на образование'
credit_score['purpose_category'] = credit_score.apply(create_category_purpose, axis=1)
credit_score['purpose_category'].head(10)

0    на недвижимость
1      на автомобиль
2    на недвижимость
3     на образование
4         на свадьбу
5    на недвижимость
6    на недвижимость
7     на образование
8         на свадьбу
9    на недвижимость
Name: purpose_category, dtype: object

In [151]:
credit_score.head(10)

,index,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_category
0,0,1,8438,42,высшее,0,женат / замужем,0,F,сотрудник,0,253876,покупка жилья,на недвижимость
1,1,1,4025,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,на автомобиль
2,2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145886,покупка жилья,на недвижимость
3,3,3,4125,32,среднее,1,женат / замужем,0,M,сотрудник,0,267629,дополнительное образование,на образование
4,4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,на свадьбу
5,5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255764,покупка жилья,на недвижимость
6,6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240526,операции с жильем,на недвижимость
7,7,0,153,50,среднее,1,женат / замужем,0,M,сотрудник,0,135824,образование,на образование
8,8,2,6930,35,высшее,0,гражданский брак,1,F,сотрудник,0,95857,на проведение свадьбы,на свадьбу
9,9,0,2189,41,среднее,1,женат / замужем,0,M,сотрудник,0,144426,покупка жилья для семьи,на недвижимость


**Вывод** Использовал лемматизацию для выделения основные категорий обращений

### Категоризация данных

In [152]:
def create_salary_cat(row): #напишем функцию для категоризации людей по доходу. Низкий уровень дохода выберу 60000 рублей(средняя зп по Спб)
        if row['total_income'] <= 60000:
            return ' низкий уровень дохода'
        if (row['total_income'] > 60000) and (row['total_income'] <= 150000):
            return 'средний уровень дохода'
        elif row['total_income'] > 150000:
            return 'высокий уровень дохода'
credit_score['salary_cat'] = credit_score.apply(create_salary_cat, axis=1)
credit_score['salary_cat'].head(10)

0    высокий уровень дохода
1    средний уровень дохода
2    средний уровень дохода
3    высокий уровень дохода
4    высокий уровень дохода
5    высокий уровень дохода
6    высокий уровень дохода
7    средний уровень дохода
8    средний уровень дохода
9    средний уровень дохода
Name: salary_cat, dtype: object

In [153]:
def create_children_cat(row): 
    if row['children'] == 0:
        return 'нет детей'
    if 1 <= row['children'] <= 2:
        return '1-2 ребенка'
    else:
        return 'многодетные'
credit_score['children_cat'] = credit_score.apply(create_children_cat, axis=1)
credit_score['children_cat'].head(10)

0    1-2 ребенка
1    1-2 ребенка
2      нет детей
3    многодетные
4      нет детей
5      нет детей
6      нет детей
7      нет детей
8    1-2 ребенка
9      нет детей
Name: children_cat, dtype: object

**Вывод** Категоризировал данные по доходу и добавил в таблицу

## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [154]:
#print(credit_score['children'].value_counts()) #  найдем количество детей
#у 76 человек 20 детей. Можно заменить на медианное значение по остальным столбцам
# так же есть отрицательное количество детей?? скорее всего была ошибка в знаке
#сначала методом abs вернем по модулю и проверим, затем заменяем на медианное значение по остальным столбцам
credit_score['children'] = credit_score['children'].apply(abs)
print(credit_score['children'].value_counts())

0     14091
1      4855
2      2052
3       330
20       76
4        41
5         9
Name: children, dtype: int64


In [155]:
children_median = credit_score[credit_score['children'] != 20]['children'].median().astype('int')
credit_score['children'] = credit_score['children'].replace(20, children_median) #замену произвели, данные готовы к обработке


In [185]:
table_child_cat = credit_score.pivot_table(index='children_cat', columns= 'debt', values='children', aggfunc='count')
frmt = '{:.1%}'.format
table_child_cat['процент'] = (table_child_cat[1] / (table_child_cat[1] + table_child_cat[0])).map(frmt)
#display(table_child_cat.sort_index(ascending = False)
display(table_child_cat.sort_index(ascending=False))

debt,0,1,процент
children_cat,,,
нет детей,13028,1063,7.5%
многодетные,463,40,8.0%
1-2 ребенка,6222,638,9.3%


**Вывод** Исходя из полученной таблицы, заёмщики с несколькими детьми возвращают деньги реже. Процент с несколькими детьми выше, но не большой. 

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [157]:
credit_score = credit_score.replace('гражданский брак', 'женат / замужем') #заменяем значения
credit_score.reset_index(drop=True) #обновляем индексацию
print(credit_score['family_status'].value_counts())
famil_conversion = (credit_score[credit_score['debt'] == 1].groupby('family_status')['debt'].count() / credit_score.groupby('family_status')['debt'].count()).round(3)
famil_conversion.to_frame('conversion table') #посчитали конверсию 

женат / замужем          16490
Не женат / не замужем     2810
в разводе                 1195
вдовец / вдова             959
Name: family_status, dtype: int64


,conversion table
family_status,
Не женат / не замужем,0.098
в разводе,0.071
вдовец / вдова,0.066
женат / замужем,0.080


**Вывод**  Исходя из конверсии женатые люди платят лучше остальных групп. 

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [158]:
def conclusion(category): # создал фукцию для вывода информации 
    return credit_score.groupby(category)['debt'].mean().to_frame().sort_values(by='debt')

In [159]:
conclusion('salary_cat')

,debt
salary_cat,
низкий уровень дохода,0.060794
высокий уровень дохода,0.077612
средний уровень дохода,0.085883


**Вывод** Забавный вывод. Люди с доходом меньше 60000 менее склонны нарушать обязательства по выплатам кредита в срок

- Как разные цели кредита влияют на его возврат в срок?

In [160]:
conclusion('purpose_category')

,debt
purpose_category,
на недвижимость,0.072334
на свадьбу,0.080034
на образование,0.092200
на автомобиль,0.093590


**Вывод**Заемщики связанные с категорией жилья наиболее ответственные и менее сколнны просрачивать платежы

## Шаг 4. Общий вывод

**Вывод**

Общий вывод будет таковым:
1) Заёмщики состоящие в браке и не имеющие детей - наиболее сколнны к своевременным платежам и реже пропускают платежы
2) Заёмщики, состоящие в гражданском браке или без отношений - самые менее ответственные заёмщики.